In [ ]:
import os

#os.environ["HF_DATASETS_CACHE"] = "/domino/datasets/local/SAS-R/"
os.environ["TRANSFORMERS_CACHE"] =  "/mnt/data/CodeGen"

In [ ]:
from transformers.utils import TRANSFORMERS_CACHE
TRANSFORMERS_CACHE

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

In [ ]:
model_name = 'codellama/CodeLlama-7b-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             cache_dir=TRANSFORMERS_CACHE,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                            )


In [5]:
model.save_pretrained(TRANSFORMERS_CACHE)

In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer
               )

In [12]:
code="""
/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;
"""

In [13]:
convert="You are a code conversion tool. You will be given SAS code, please convert it to R code.\n"
thisPrompt="{}\n{}".format(convert, code)

In [14]:
sequences = pipe(
    thisPrompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=400
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: You are a code conversion tool. You will be given SAS code, please convert it to R code.


/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;


/* Load the input dataset */
cars <- read.csv("/mnt/data/mtcars.csv")

/* Sort the input dataset by displacement */
cars <- cars[order(cars$disp),]

/* Perform the linear regress